In [12]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
#from scipy.signal import argrelextrema
from plotly.subplots import make_subplots

# load in data
trad2v = pd.read_csv("../Data/Trades2Vis/LongStrat10k240.csv")

trad2v['entry-time'] = pd.to_datetime(trad2v['entry-time'], utc=True)
trad2v['exit-time'] = pd.to_datetime(trad2v['exit-time'], utc=True)




    # rule_str = 'lambda x, i: tuple_(lt_(get_(x, i, "RSI"), 30), gt_(get_(x, i, "RSI"), 70))'
    # rule = eval(rule_str)
    # total_profit, ftrades = TradeSim(rule, data)
    # print(len(ftrades))
    # print(total_profit)

    # program_tree = [
    #     'tuple_',
    #     ['lt_', 'x.RSI', '30'],
    #     ['gt_', 'x.RSI', '70']
    # ]
    
    # test_rule = TradingStrategy(program=program_tree)

    # # Compute likelihood and see trades
    # log_lik = test_rule.compute_likelihood(wdata)
    # profit = np.exp(log_lik)
    # print("Log-likelihood:", log_lik)
    # print("Profit:", profit)
    # print("Trades:", test_rule.trades)
    # steps = 10000
    # tell = [samp for samp in range(0, steps, int(steps/100))]


In [13]:
## Code to plot the trades on price data
priceData = pd.read_csv("../Data/PriceData/KRAKEN_ADAUSD, 240.csv")

# get the last and top strategy
strat = trad2v[trad2v['Strategy'] == 9]

# Create figure with 2 rows: main price chart + RSI subplot
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.04,
    row_heights=[0.6, 0.25, 0.15],
    subplot_titles=["Price with Trades and Bollinger Bands", "RSI and RSI-based MA", "Profit Over Time"]
)

# create a candlestick plot to show price
fig.add_trace(go.Candlestick(
    x=priceData.time,
    open=priceData.open,
    high=priceData.high,
    low=priceData.low,
    close=priceData.close,
    name='Price'
    ), row=1, col=1)

# # Add major low markers (triangle-up, green)
# fig.add_trace(go.Scatter(
#     x=major_lows['time'],
#     y=major_lows['close'],
#     mode='markers',
#     marker=dict(symbol='triangle-up', color='green', size=10),
#     name='Major Low'
# ))

# # Add major high markers (triangle-down, red)
# fig.add_trace(go.Scatter(
#     x=major_highs['time'],
#     y=major_highs['close'],
#     mode='markers',
#     marker=dict(symbol='triangle-down', color='red', size=10),
#     name='Major High'
# ))

# --- Bollinger Bands ---
fig.add_trace(go.Scatter(
    x=priceData['time'],
    y=priceData['Upper'],
    line=dict(color='rgba(255, 0, 0, 0.3)', width=1),
    name='Upper Band'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=priceData['time'],
    y=priceData['Basis'],
    line=dict(color='rgba(0, 0, 255, 0.4)', width=1),
    name='Basis'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=priceData['time'],
    y=priceData['Lower'],
    line=dict(color='rgba(0, 255, 0, 0.3)', width=1),
    name='Lower Band'
), row=1, col=1)

# --- RSI and RSI MA Subplot ---
fig.add_trace(go.Scatter(
    x=priceData['time'],
    y=priceData['RSI'],
    line=dict(color='orange', width=1),
    name='RSI'
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=priceData['time'],
    y=priceData['RSI-based MA'],
    line=dict(color='purple', width=1),
    name='RSI MA'
), row=2, col=1)

# Add overbought/oversold lines
fig.add_shape(type='line', x0=priceData['time'].min(), x1=priceData['time'].max(),
              y0=70, y1=70, line=dict(color='red', dash='dash'), row=2, col=1)
fig.add_shape(type='line', x0=priceData['time'].min(), x1=priceData['time'].max(),
              y0=30, y1=30, line=dict(color='green', dash='dash'), row=2, col=1)

# Add shapes and annotations per trade
for _, trade in strat.iterrows():
    
    entry_time = trade['entry-time']
    exit_time = trade['exit-time']
    entry_price = trade['entry-price']
    exit_price = trade['exit-price']
    pct_change = trade['pct_change']
    profit = trade['profit']

    low_price = min(entry_price, exit_price)
    high_price = max(entry_price, exit_price)

    # 1. Rectangle for trade range
    fig.add_shape(
        type="rect",
        x0=entry_time,
        x1=exit_time,
        y0=low_price,
        y1=high_price,
        fillcolor="rgba(0, 0, 255, 0.1)",
        line=dict(width=0),
        layer="below"
    )

    # 2. Horizontal dotted green line at entry price
    fig.add_shape(
        type="line",
        x0=entry_time,
        x1=exit_time,
        y0=entry_price,
        y1=entry_price,
        line=dict(color="green", dash="dot", width=1.5),
        layer="above"
    )

    # 3. Horizontal dotted red line at exit price
    fig.add_shape(
        type="line",
        x0=entry_time,
        x1=exit_time,
        y0=exit_price,
        y1=exit_price,
        line=dict(color="red", dash="dot", width=1.5),
        layer="above"
    )

    # 4. Vertical dotted lines for timing
    fig.add_shape(
        type="line",
        x0=entry_time,
        x1=entry_time,
        y0=low_price,
        y1=high_price,
        line=dict(color="green", dash="dot", width=1.2),
        layer="above"
    )
    fig.add_shape(
        type="line",
        x0=exit_time,
        x1=exit_time,
        y0=low_price,
        y1=high_price,
        line=dict(color="red", dash="dot", width=1.2),
        layer="above"
    )

    # 5. Invisible marker to show trade info on hover
    mid_time = entry_time  # (optionally: midpoint = entry + (exit - entry)/2)
    mid_price = (entry_price + exit_price) / 2

    hover_text = (
        f"Entry Date: {entry_time}<br>"
        f"Entry Price: {entry_price:.4f}<br>"
        f"Exit Date: {exit_time}<br>"
        f"Exit Price: {exit_price:.4f}<br>"
        f"Change: {pct_change:.2f}%<br>"
        f"Profit: {profit:.2f}"
    )

    fig.add_trace(go.Scatter(
        x=[mid_time],
        y=[mid_price],
        mode="markers",
        marker=dict(size=1, color="rgba(0,0,0,0)"),
        hoverinfo="text",
        hovertext=hover_text,
        showlegend=False
    ))

# --- Profit over time subplot ---
fig.add_trace(go.Scatter(
    x=strat['exit-time'],
    y=strat['profit'].cumsum(),
    mode='lines+markers',
    line=dict(color='green', width=2),
    name='Cumulative Profit'
), row=3, col=1)

# Final layout
fig.update_layout(
    title='Stock Price with Bollinger Bands and RSI',
    xaxis_rangeslider_visible=False,
    hovermode='x unified',
    height=800
)



fig.show()